In [ ]:
# CÉLULA PARA O ARQUIVO: 05-Sistema.ipynb
# Responsabilidade: Unir todas as partes e apresentar o sistema interativo final.

import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore

load_dotenv()

# --- Configurações Iniciais ---
llm = ChatGroq(temperature=0, model_name="gemma2-9b-it")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
PINECONE_INDEX_NAME = os.environ.get("PINECONE_INDEX_NAME")
vectorstore = PineconeVectorStore(index_name=PINECONE_INDEX_NAME, embedding=embeddings)

# --- Cadeias Especializadas ---
prompt_roteiro = PromptTemplate.from_template("Crie um roteiro detalhado para: {input}")
chain_roteiro = prompt_roteiro | llm | StrOutputParser()

prompt_logistica = PromptTemplate.from_template("Responda à pergunta de logística: {input}")
chain_logistica = prompt_logistica | llm | StrOutputParser()

chain_info_local = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

# --- Roteador ---
router_template = "Classifique a pergunta em: 'roteiro-viagem', 'info-local', 'logistica-transporte'. Pergunta: {input}"
prompt_router = PromptTemplate.from_template(router_template)
chain_classificadora = prompt_router | llm | StrOutputParser()

# --- Loop de Execução ---
print("🤖 Olá! Sou seu assistente de viagens. Digite 'sair' para encerrar.")

while True:
    user_input = input("\nSua pergunta: ")
    if user_input.lower() == 'sair':
        break

    topico = chain_classificadora.invoke({"input": user_input})
    print(f"DEBUG: Tópico identificado -> {topico}")

    if "roteiro-viagem" in topico:
        response = chain_roteiro.invoke({"input": user_input})
        print("\nResposta:", response)
    elif "info-local" in topico:
        response = chain_info_local.invoke({"query": user_input})
        print("\nResposta:", response['result'])
    elif "logistica-transporte" in topico:
        response = chain_logistica.invoke({"input": user_input})
        print("\nResposta:", response)
    else:
        response = chain_info_local.invoke({"query": user_input})
        print("\nResposta (rota padrão):", response['result'])